In [1]:
import sys,os
sys.path.append("..")

import numpy as np
import tensorflow as tf
from example import bert_classifier
from bunch import Bunch
from example import feature_writer, write_to_tfrecords, classifier_processor
from data_generator import tokenization
from data_generator import tf_data_utils
from model_io import model_io
import json

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_excel("/data/xuht/eventy_detection/sentiment/model/output.xlsx", sheetname="Sheet2")
data_set = []
data_label = []
for index in range(data.shape[0]):
    data_set.append(data.loc[index]["title"])
    data_label.append(data.loc[index]["sentiment"])
X_train, X_test, y_train, y_test = train_test_split(data_set, data_label, test_size=0.2, random_state=42)

with open("/data/xuht/eventy_detection/sentiment/model/test/train.txt", "w") as fwobj:
    for item, label in zip(X_train, y_train):
        fwobj.write(" ".join([item, "__label__{}".format(str(label))])+"\n")
with open("/data/xuht/eventy_detection/sentiment/model/test/test.txt", "w") as fwobj:
    for item, label in zip(X_test, y_test):
        fwobj.write(" ".join([item, "__label__{}".format(str(label))])+"\n")
import json
with open("/data/xuht/eventy_detection/sentiment/model/test/label2id.json", "w") as fwobj:
    json.dump({"label2id":{"0":1, "1":2, "-1":0}, "id2label":{0:"-1", 1:"0", 2:"1"}}, fwobj)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_excel("/data/xuht/eventy_detection/sentiment/model/sentiment_11_14.xlsx", sheetname="Sheet1")
data_set = []
data_label = []
for index in range(data.shape[0]):
    data_set.append(data.loc[index]["句子内容"])
    data_label.append(data.loc[index]["预期情感"])
X_train, X_test, y_train, y_test = train_test_split(data_set, data_label, test_size=0.2, random_state=42)

with open("/data/xuht/eventy_detection/sentiment/model/bert/train_11_15.txt", "w") as fwobj:
    for item, label in zip(X_train, y_train):
        fwobj.write(" ".join([item, "__label__{}".format(str(label))])+"\n")
with open("/data/xuht/eventy_detection/sentiment/model/bert/test_11_15.txt", "w") as fwobj:
    for item, label in zip(X_test, y_test):
        fwobj.write(" ".join([item, "__label__{}".format(str(label))])+"\n")
import json
with open("/data/xuht/eventy_detection/sentiment/model/bert/label2id.json", "w") as fwobj:
    json.dump({"label2id":{"0":1, "1":2, "-1":0}, "id2label":{0:"-1", 1:"0", 2:"1"}}, fwobj)

In [ ]:
train = []
tokenizer = tokenization.FullTokenizer(
      vocab_file="/data/xuht/bert/chinese_L-12_H-768_A-12/vocab.txt", 
    do_lower_case=True)
label_mapping = {"label2id":{"good":1, "bad":0}, "id2label":{0:"bad", 1:"good"}}
json.dump(label_mapping, open("/data/xuht/jd_comment/label.json", "w"))
with open("/data/xuht/jd_comment/train.tsv", "r") as frobj:
    for line in frobj:
        content = line.strip().split("\t")
        label = content[1]
        sent = content[-1]
        train.append([sent, label])

import random
random.shuffle(train)

dev = []
with open("/data/xuht/jd_comment/dev.tsv", "r") as frobj:
    for line in frobj:
        content = line.strip().split("\t")
        label = content[1]
        sent = content[-1]
        dev.append([sent, label])
        
random.shuffle(dev)

left_train = []
with open("/data/xuht/jd_comment/train.txt", "w") as fwobj:
    for item in train:
        if item not in left_train:
            fwobj.write(" ".join([item[0], "__label__{}".format(str(item[1]))])+"\n")
            left_train.append(item)
left_dev = []
with open("/data/xuht/jd_comment/test.txt", "w") as fwobj:
    for item in dev:
        if item not in left_dev:
            fwobj.write(" ".join([item[0], "__label__{}".format(str(item[1]))])+"\n")
            left_dev.append(item)
classifier_data_api = classifier_processor.ClassificationProcessor()

classifier_data_api.get_labels("/data/xuht/jd_comment/label.json")
train_examples = classifier_data_api.get_train_examples("/data/xuht/jd_comment/train.txt")
test_examples = classifier_data_api.get_train_examples("/data/xuht/jd_comment/test.txt")

write_to_tfrecords.convert_classifier_examples_to_features(train_examples,classifier_data_api.label2id,
                                                           128,
                                                           tokenizer,
                                                           "/data/xuht/jd_comment/train.tfrecords"
                                                          )
write_to_tfrecords.convert_classifier_examples_to_features(test_examples,classifier_data_api.label2id,
                                                           128,
                                                           tokenizer,
                                                           "/data/xuht/jd_comment/test.tfrecords"
                                                          )

In [ ]:
train_examples[0:5]

In [ ]:
import jieba
import pandas as pd
data = pd.read_excel("/data/xuht/eventy_detection/data_1114.xlsx")
with open("/data/xuht/general_dict/Company-Names-Corpus.txt", "r") as frobj:
    company = frobj.read().splitlines()[3:]
with open("/data/xuht/general_dict/Chinese_Names_Corpus.txt", "r") as frobj:
    name = frobj.read().splitlines()[3:]
with open("/data/xuht/Chinese_w2v/Tencent_AILab_ChineseEmbedding/vocab.txt") as frobj:
    vocab = frobj.read().splitlines()
    
for item in company:
    jieba.add_word(item, 10000, "<compnay>")
for item in name:
    jieba.add_word(item, 10000, "<person>")
for item in vocab:
    jieba.add_word(item, 10000)


In [ ]:
import pandas as pd
data = pd.read_excel("/data/xuht/eventy_detection/data_1114.xlsx")
with open("/data/xuht/eventy_detection/stopwords-master/ltp_stop.txt", "r") as frobj:
    stop_word = frobj.read().splitlines()
import jieba.posseg as pseg
import re
char_pattern = re.compile(u"[\u4e00-\u9fa5]+")
import html, re

def replace(doc):
    doc = re.sub(u"\\（.*?）|\\{.*?}|\\[.*?]|\\【.*?】|\\(.*?\\)|\\〔.*?〕", "", doc)
    doc = re.sub(r'\d?.?\d+.?\d*(%|百分点)', '<percent>', doc)
    doc = re.sub(r'(（|\()\d+.?\w*(）|\))', '<stock_no>', doc)
    doc = re.sub(r'\d+.?\d*(\w元|百万|千万|亿|万)', '<money>', doc)
    doc = re.sub(r'\d+.?\d+(\d|,)*?(股|\w股)', '<stock_num>', doc)
    doc = re.sub(r'\d+.?\d*\w?(千瓦\w?|\w吨)', '<energe_num>', doc)
    doc = re.sub(r'(\d+年)?\d+月\d+(日|号)', '<date>', doc)
    doc = re.sub(r'\d+年\d+月', '<date>', doc)
    doc = re.sub(r'\d+年-\d+年', '<date>', doc)
    doc = re.sub(r'\d+年', '<date>', doc)
    doc = re.sub(u"\\《.*?》", "<title>", doc)
    doc = re.sub(u"\d+、", "", doc)

    return doc

def full2half(s):
    n = []
    for char in s:
        num = ord(char)
        if num == 0x3000:
            num = 32
        elif 0xFF01 <= num <= 0xFF5E:
            num -= 0xfee0
        num = chr(num)
        n.append(num)
    return ''.join(n)

def normal(doc):
    doc = doc.lower()
    doc = full2half(doc)

    regx = re.compile(r'(http://|www){1}[a-zA-Z0-9.?/&=:]*',re.S) 
    doc = regx.sub("", doc)

    doc = html.unescape(doc).replace(' ', '') 
    doc = replace(doc)
    return doc

for t in ["<title>", "<date>", "<energe_num>", "<money>", "<stock_num>", "<percent>", "<stock_no>"]:
    jieba.add_word(t, 10000)

sent_key = {}
for index in range(data.shape[0]):
    if data.loc[index]["句子内容"] in sent_key:
        sent_key[data.loc[index]["句子内容"]].append(data.loc[index]["预期事件"])
    else:
        sent_key[data.loc[index]["句子内容"]] = [data.loc[index]["预期事件"]]
        
real_sent = {}
for key in sent_key:
    if len(set(sent_key[key])) >= 2:
        continue
    else:
        real_sent[key] = list(set(sent_key[key]))[0]
        
data_set = [key for key in real_sent]
data_label = [real_sent[key] for key in real_sent]
from sklearn.model_selection import train_test_split

def prepare(sent):
    sent = replace(sent)
    sent = normal(sent)
    sent = full2half(sent)
    sent = normal(sent)
    word_lst_ = list(pseg.lcut(sent.lower()))
    word_lst = []
    for item in word_lst_:
        item = list(item)
        if item[0] in stop_word:
            continue
        elif item[1] in ["<compnay>", "<person>"]:
            word_lst.append(item[1])
        else:
            if len(re.findall(char_pattern, item[0])) >= 1:
                for sub_item in item[0]:
                    if len(sub_item) >= 1:
                        word_lst.append(sub_item)
            else:
                if len(item[0]) >= 1:
                    word_lst.append(item[0])
    return " ".join(word_lst)

X_train, X_test, y_train, y_test = train_test_split(data_set, data_label, test_size=0.2, random_state=42)
label_cnt = {}
for item in y_train:
    if item in label_cnt:
        label_cnt[item] += 1
    else:
        label_cnt[item] = 1
valid_label = [label for label in label_cnt if label_cnt[label] >= 100 and label != "资产划入" and label != "资产划出"]

label_mapping = {}
for label in label_cnt:
    if label not in valid_label:
        label_mapping[label] = "other"
    else:
        label_mapping[label] = label

with open("/data/xuht/eventy_detection/event/model/train_11_15.txt", "w") as fwobj:
    for item, label in zip(X_train, y_train):
        item = full2half(item)
        l = label_mapping[label]
#             item = prepare(item)
        fwobj.write(" ".join([item, "__label__{}".format(str(l))])+"\n")
with open("/data/xuht/eventy_detection/event/model/test_11_15.txt", "w") as fwobj:
    for item, label in zip(X_test, y_test):
        l = label_mapping[label]
        item = full2half(item)
#             item = prepare(item)
        fwobj.write(" ".join([item, "__label__{}".format(str(l))])+"\n")

label2id = {}
id2label = {}
label = set([label_mapping[key] for key in label_mapping])
for index, l in enumerate(label):
    label2id[l] = index
    id2label[index] = l
import json
with open("/data/xuht/eventy_detection/event/model/label2id.json", "w") as fwobj:
    json.dump({"label2id":label2id, "id2label":id2label}, fwobj)

In [ ]:
with open("/data/xuht/label_text.txt", "w") as fwobj:
    for key in label2id:
        fwobj.write(key+"\n")

In [ ]:
tokenizer = tokenization.FullTokenizer(
      vocab_file="/data/xuht/bert/chinese_L-12_H-768_A-12/vocab.txt", 
    do_lower_case=True)

In [ ]:
classifier_data_api = classifier_processor.ClassificationProcessor()

classifier_data_api.get_labels("/data/xuht/eventy_detection/event/model/label2id.json")
train_examples = classifier_data_api.get_train_examples("/data/xuht/eventy_detection/event/model/train_11_15.txt")
test_examples = classifier_data_api.get_train_examples("/data/xuht/eventy_detection/event/model/test_11_15.txt")

write_to_tfrecords.convert_classifier_examples_to_features(train_examples,classifier_data_api.label2id,
                                                           100,
                                                           tokenizer,
                                                           "/data/xuht/eventy_detection/event/model/train.tfrecords"
                                                          )
write_to_tfrecords.convert_classifier_examples_to_features(test_examples,classifier_data_api.label2id,
                                                           100,
                                                           tokenizer,
                                                           "/data/xuht/eventy_detection/event/model/test.tfrecords"
                                                          )

In [ ]:
train_examples[0]

In [ ]:
classifier_data_api = classifier_processor.ClassificationProcessor()
classifier_data_api.get_labels("/data/xuht/eventy_detection/sentiment/model/sentiment_11_14/label2id.json")
train_examples = classifier_data_api.get_train_examples("/data/xuht/eventy_detection/sentiment/model/test/train.txt")
test_examples = classifier_data_api.get_train_examples("/data/xuht/eventy_detection/sentiment/model/test/test.txt")

print(len(train_examples))

write_to_tfrecords.convert_classifier_examples_to_features(train_examples,classifier_data_api.label2id,
                                                           100,
                                                           tokenizer,
                                                           "/data/xuht/eventy_detection/sentiment/model/test/train.tfrecords"
                                                          )
write_to_tfrecords.convert_classifier_examples_to_features(test_examples,classifier_data_api.label2id,
                                                           100,
                                                           tokenizer,
                                                           "/data/xuht/eventy_detection/sentiment/model/test/test.tfrecords"
                                                          )

In [ ]:
classifier_data_api = classifier_processor.ClassificationProcessor()
classifier_data_api.get_labels("/data/xuht/eventy_detection/sentiment/model/bert/label2id.json")
train_examples = classifier_data_api.get_train_examples("/data/xuht/eventy_detection/sentiment/model/bert/train_11_15.txt")
test_examples = classifier_data_api.get_train_examples("/data/xuht/eventy_detection/sentiment/model/bert/test_11_15.txt")

print(len(train_examples))

write_to_tfrecords.convert_classifier_examples_to_features(train_examples,classifier_data_api.label2id,
                                                           100,
                                                           tokenizer,
                                                           "/data/xuht/eventy_detection/sentiment/model/bert/train_11_15.tfrecords"
                                                          )
write_to_tfrecords.convert_classifier_examples_to_features(test_examples,classifier_data_api.label2id,
                                                           100,
                                                           tokenizer,
                                                           "/data/xuht/eventy_detection/sentiment/model/bert/test_11_15.tfrecords"
                                                          )

In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

graph = tf.Graph()
with graph.as_default():
    import json
    
    config = json.load(open("/data/xuht/bert/chinese_L-12_H-768_A-12/bert_config.json", "r"))
#     init_checkpoint = "/data/xuht/bert/chinese_L-12_H-768_A-12/bert_model.ckpt"
    init_checkpoint = "/data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt"
    config = Bunch(config)
    config.use_one_hot_embeddings = True
    config.scope = "bert"
    config.dropout_prob = 0.1
    config.label_type = "single_label"
#     config.num_hidden_layers = 
    
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    
    num_train_steps = int(
        33033 / 32 * 2)
    num_warmup_steps = int(num_train_steps * 0.1)
    sess = tf.Session()
    
    opt_config = Bunch({"init_lr":1e-5, "num_train_steps":num_train_steps, "cycle":False, 
                        "num_warmup_steps":num_warmup_steps,
                       "lr_decay":"polynomial_decay"})
    model_io_config = Bunch({"fix_lm":False})
    
    model_io_fn = model_io.ModelIO(model_io_config)
    
    num_calsses = 3

    model_train_fn = bert_classifier.classifier_model_fn_builder(config, num_calsses, init_checkpoint, 
                                            reuse=None, 
                                            load_pretrained=True,
                                            model_io_fn=model_io_fn,
                                            model_io_config=model_io_config, 
                                            opt_config=opt_config)
    
    model_eval_fn = bert_classifier.classifier_model_fn_builder(config, num_calsses, init_checkpoint, 
                                            reuse=True, 
                                            load_pretrained=True,
                                            model_io_fn=model_io_fn,
                                            model_io_config=model_io_config, 
                                            opt_config=opt_config)
    
    def metric_fn(features, logits, loss):
        print(logits.get_shape(), "===logits shape===")
        pred_label = tf.argmax(logits, axis=-1, output_type=tf.int32)
        prob = tf.nn.softmax(logits)
        accuracy = correct = tf.equal(
            tf.cast(pred_label, tf.int32),
            tf.cast(features["label_ids"], tf.int32)
        )
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        return {"accuracy":accuracy, "loss":loss, "pred_label":pred_label, "label_ids":features["label_ids"]}
    
    name_to_features = {
            "input_ids":
                    tf.FixedLenFeature([128], tf.int64),
            "input_mask":
                    tf.FixedLenFeature([128], tf.int64),
            "segment_ids":
                    tf.FixedLenFeature([128], tf.int64),
            "label_ids":
                    tf.FixedLenFeature([], tf.int64),
    }

    params = Bunch({})
    params.epoch = 2
    params.batch_size = 32
    train_file = "/data/xuht/eventy_detection/event/model/train.tfrecords"
    train_file1 = "/data/xuht/eventy_detection/sentiment/model/sentiment_11_14/train.tfrecords"
    title_sentiment = "/data/xuht/eventy_detection/sentiment/model/test/train.tfrecords"
    sentiment = "/data/xuht/eventy_detection/sentiment/model/bert/train_11_15.tfrecords"
    jd_train = "/data/xuht/jd_comment/train.tfrecords"
    train_features = tf_data_utils.train_input_fn(jd_train,
                                tf_data_utils._decode_record, name_to_features, params)
    
    test_file = ["/data/xuht/eventy_detection/sentiment/model/sentiment_11_14/test.tfrecords"]
    test_file1_1 = ["/data/xuht/eventy_detection/sentiment/model/test/train.tfrecords",
                "/data/xuht/eventy_detection/sentiment/model/test/test.tfrecords"]
    test_file2 = "/data/xuht/eventy_detection/event/model/test.tfrecords"
    title_test = "/data/xuht/eventy_detection/sentiment/model/test/test.tfrecords"
    jd_test = "/data/xuht/jd_comment/test.tfrecords"
    sentiment_test = "/data/xuht/eventy_detection/sentiment/model/bert/test_11_15.tfrecords"
    
    eval_features = tf_data_utils.eval_input_fn(jd_test,
                                tf_data_utils._decode_record, name_to_features, params)
    
    [train_op, train_loss, train_per_example_loss, train_logits] = model_train_fn(train_features, [], tf.estimator.ModeKeys.TRAIN)
    [_, eval_loss, eval_per_example_loss, eval_logits] = model_eval_fn(eval_features, [], tf.estimator.ModeKeys.EVAL)
    result = metric_fn(eval_features, eval_logits, eval_loss)
    
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)
    
    model_io_fn.set_saver()
    
    def eval_fn(result):
        i = 0
        total_accuracy = 0
        label, label_id = [], []
        while True:
            try:
                eval_result = sess.run(result)
                total_accuracy += eval_result["accuracy"]
                label_id.extend(eval_result["label_ids"])
                label.extend(eval_result["pred_label"])
                i += 1
            except tf.errors.OutOfRangeError:
                print("End of dataset")
                break
        macro_f1 = f1_score(label_id, label, average="macro")
        micro_f1 = f1_score(label_id, label, average="micro")
        accuracy = accuracy_score(label_id, label)
        print("test accuracy {} macro_f1 score {} micro_f1 {} accuracy {}".format(total_accuracy/ i, 
                                                                                macro_f1,  micro_f1, accuracy))
        return total_accuracy/ i, label_id, label
    
    def train_fn(op, loss):
        i = 0
        total_loss = 0
        cnt = 0
        while True:
            try:
                [_, train_loss] = sess.run([op, loss])
                i += 1
                cnt += 1
                total_loss += train_loss
                if np.mod(i, 1000) == 0:
                    print(total_loss/cnt)
                    cnt = 0
                    total_loss = 0
            except tf.errors.OutOfRangeError:
                print("End of dataset")
                break
            
    train_fn(train_op, train_loss)
    acc, true_label, pred_label = eval_fn(result)
#     model_io_fn.save_model(sess, "/data/xuht/eventy_detection/sentiment/model/bert/sentiment.ckpt")

 initializing ModelIO 
{'input_ids': <tf.Tensor 'IteratorGetNext:0' shape=(?, 128) dtype=int32>, 'label_ids': <tf.Tensor 'IteratorGetNext:2' shape=(?,) dtype=int32>, 'segment_ids': <tf.Tensor 'IteratorGetNext:3' shape=(?, 128) dtype=int32>, 'input_mask': <tf.Tensor 'IteratorGetNext:1' shape=(?, 128) dtype=int32>}
INFO:tensorflow: begin to build bert
INFO:tensorflow:Initialize variable bert/embeddings/LayerNorm/beta:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/embeddings/LayerNorm/beta
INFO:tensorflow:Initialize variable bert/embeddings/LayerNorm/gamma:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/embeddings/LayerNorm/gamma
INFO:tensorflow:Initialize variable bert/embeddings/position_embeddings:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/embeddings/position_embeddings
INFO:tensorflow:Initialize variable bert/embeddings/token_type_embeddings:0 from 

INFO:tensorflow:Initialize variable bert/encoder/layer_10/attention/output/LayerNorm/beta:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/encoder/layer_10/attention/output/LayerNorm/beta
INFO:tensorflow:Initialize variable bert/encoder/layer_10/attention/output/LayerNorm/gamma:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/encoder/layer_10/attention/output/LayerNorm/gamma
INFO:tensorflow:Initialize variable bert/encoder/layer_10/attention/output/dense/bias:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/encoder/layer_10/attention/output/dense/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_10/attention/output/dense/kernel:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/encoder/layer_10/attention/output/dense/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_10/attention/self/key/bias:0 fr

INFO:tensorflow:Initialize variable bert/encoder/layer_2/attention/self/query/bias:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/encoder/layer_2/attention/self/query/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_2/attention/self/query/kernel:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/encoder/layer_2/attention/self/query/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_2/attention/self/value/bias:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/encoder/layer_2/attention/self/value/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_2/attention/self/value/kernel:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/encoder/layer_2/attention/self/value/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_2/intermediate/dense/bias:0 from checkpoint /data/xuht/oqmrc/concat/d

INFO:tensorflow:Initialize variable bert/encoder/layer_4/output/LayerNorm/beta:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/encoder/layer_4/output/LayerNorm/beta
INFO:tensorflow:Initialize variable bert/encoder/layer_4/output/LayerNorm/gamma:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/encoder/layer_4/output/LayerNorm/gamma
INFO:tensorflow:Initialize variable bert/encoder/layer_4/output/dense/bias:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/encoder/layer_4/output/dense/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_4/output/dense/kernel:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/encoder/layer_4/output/dense/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_5/attention/output/LayerNorm/beta:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt wit

INFO:tensorflow:Initialize variable bert/encoder/layer_7/attention/output/dense/bias:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/encoder/layer_7/attention/output/dense/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_7/attention/output/dense/kernel:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/encoder/layer_7/attention/output/dense/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_7/attention/self/key/bias:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/encoder/layer_7/attention/self/key/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_7/attention/self/key/kernel:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/encoder/layer_7/attention/self/key/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_7/attention/self/query/bias:0 from checkpoint /data/xuht/oqmrc/concat

INFO:tensorflow:Initialize variable bert/encoder/layer_9/attention/self/value/bias:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/encoder/layer_9/attention/self/value/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_9/attention/self/value/kernel:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/encoder/layer_9/attention/self/value/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_9/intermediate/dense/bias:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/encoder/layer_9/intermediate/dense/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_9/intermediate/dense/kernel:0 from checkpoint /data/xuht/oqmrc/concat/data/model_11_26_distributed/oqmrc.ckpt with bert/encoder/layer_9/intermediate/dense/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_9/output/LayerNorm/beta:0 from checkpoint /data/xuht/oqmrc/concat/data/model_

ValueError: Shape of variable bert/output_weights:0 ((3, 768)) doesn't match with shape of tensor bert/output_weights ([768]) from checkpoint reader.

In [ ]:
from sklearn.metrics import classification_report
l = classification_report(true_label,pred_label)
print(l)

In [ ]:
with tf.variable_scope("teacher/bert", reuse=None): 
    inputs = tf.placeholder(tf.float32, [None, 100])
    b = tf.layers.dense(inputs, 100)
    tvars = tf.trainable_variables()
    for item in tvars:
        print(item)

In [ ]:
init_vars = tf.train.list_variables("/data/xuht/bert/chinese_L-12_H-768_A-12/bert_model.ckpt")
for x in init_vars:
    print(x[0], x[1])

In [ ]:
with graph.as_default():
    test_file1_1 = ["/data/xuht/eventy_detection/sentiment/model/test/train.tfrecords",
                "/data/xuht/eventy_detection/sentiment/model/test/test.tfrecords"]
    
    eval_features = tf_data_utils.eval_input_fn(test_file,
                                tf_data_utils._decode_record, name_to_features, params)
    
    [_, eval_loss, eval_per_example_loss, eval_logits] = model_eval_fn(eval_features, [], tf.estimator.ModeKeys.EVAL)
    result = metric_fn(eval_features, eval_logits, eval_loss)
    
    

In [ ]:
"/".join([ "bert/encoder/layer_9/output/dense/kernel", ""])

In [ ]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer

train_set = []
with open("/data/xuht/eventy_detection/sentiment/model/sentiment_11_14/train.txt", "r") as frobj:
    train = frobj.read().splitlines()
train_corpus = [item.split("__label__")[0] for item in train]
train_label = [int(item.split("__label__")[1]) for item in train]
    
with open("/data/xuht/eventy_detection/sentiment/model/sentiment_11_14/test.txt", "r") as frobj:
    test = frobj.read().splitlines()
test_corpus = [item.split("__label__")[0] for item in test]
test_label = [int(item.split("__label__")[1]) for item in test]
# vectorizer = TfidfVectorizer( ngram_range=(1, 3),max_df=0.90, min_df=2,
#                                    max_features=2000,
#                             analyzer="char")

In [ ]:
with open("/data/xuht/eventy_detection/stopwords-master/ltp_stop.txt", "r") as frobj:
    stop_word = frobj.read().splitlines()

In [ ]:
import html, re
def replace(doc):
    doc = re.sub(u"\\（.*?）|\\{.*?}|\\[.*?]|\\【.*?】|\\(.*?\\)|\\〔.*?〕", "", doc)
    doc = re.sub(r'\d?.?\d+.?\d*(%|百分点)', '<percent>', doc)
    doc = re.sub(r'(（|\()\d+.?\w*(）|\))', '<stock_no>', doc)
    doc = re.sub(r'\d+.?\d*(\w元|百万|千万|亿|万)', '<money>', doc)
    doc = re.sub(r'\d+.?\d+(\d|,)*?(股|\w股)', '<stock_num>', doc)
    doc = re.sub(r'\d+.?\d*\w?(千瓦\w?|\w吨)', '<energe_num>', doc)
    doc = re.sub(r'(\d+年)?\d+月\d+(日|号)', '<date>', doc)
    doc = re.sub(r'\d+年\d+月', '<date>', doc)
    doc = re.sub(r'\d+年-\d+年', '<date>', doc)
    doc = re.sub(r'\d+年', '<date>', doc)
    doc = re.sub(u"\\《.*?》", "<title>", doc)
    doc = re.sub(u"\d+、", "", doc)

    return doc

def full2half(s):
    n = []
    for char in s:
        num = ord(char)
        if num == 0x3000:
            num = 32
        elif 0xFF01 <= num <= 0xFF5E:
            num -= 0xfee0
        num = chr(num)
        n.append(num)
    return ''.join(n)

def normal(doc):
    doc = doc.lower()
    doc = full2half(doc)

    regx = re.compile(r'(http://|www){1}[a-zA-Z0-9.?/&=:]*',re.S) 
    doc = regx.sub("", doc)

    doc = html.unescape(doc).replace(' ', '') 
    doc = replace(doc)
    return doc
import jieba
import pandas as pd

with open("/data/xuht/general_dict/Company-Names-Corpus.txt", "r") as frobj:
    company = frobj.read().splitlines()[3:]
with open("/data/xuht/general_dict/Chinese_Names_Corpus.txt", "r") as frobj:
    name = frobj.read().splitlines()[3:]
with open("/data/xuht/Chinese_w2v/Tencent_AILab_ChineseEmbedding/vocab.txt") as frobj:
    vocab = frobj.read().splitlines()
    
for item in company:
    jieba.add_word(item, 10000, "<compnay>")
for item in name:
    jieba.add_word(item, 10000, "<person>")
for item in vocab:
    jieba.add_word(item, 10000)

In [ ]:
import numpy as np

def softmax(x):
    """
    Compute softmax values for each sets of scores in x.
    
    Rows are scores for each class. 
    Columns are predictions (samples).
    """
    scoreMatExp = np.exp(np.asarray(x))
    return scoreMatExp / scoreMatExp.sum(0)
s = softmax(np.array([ 1.9961920e-01, -4.6402103e-01, -6.1367464e-01]))
print(s.sum())

In [ ]:
p[0]["accuracy"]

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
import numpy as np
import six
import os
graph = tf.Graph()
with graph.as_default():
    import json
    
    def assert_rank(tensor, expected_rank, name=None):
        """Raises an exception if the tensor rank is not of the expected rank.

        Args:
            tensor: A tf.Tensor to check the rank of.
            expected_rank: Python integer or list of integers, expected rank.
            name: Optional name of the tensor for the error message.

        Raises:
            ValueError: If the expected shape doesn't match the actual shape.
        """
        if name is None:
            name = tensor.name

        expected_rank_dict = {}
        if isinstance(expected_rank, six.integer_types):
            expected_rank_dict[expected_rank] = True
        else:
            for x in expected_rank:
                expected_rank_dict[x] = True

        actual_rank = tensor.shape.ndims
        if actual_rank not in expected_rank_dict:
            scope_name = tf.get_variable_scope().name
            raise ValueError(
                    "For the tensor `%s` in scope `%s`, the actual rank "
                    "`%d` (shape = %s) is not equal to the expected rank `%s`" %
                    (name, scope_name, actual_rank, str(tensor.shape), str(expected_rank)))

    
    def get_shape_list(tensor, expected_rank=None, name=None):
        """Returns a list of the shape of tensor, preferring static dimensions.

        Args:
            tensor: A tf.Tensor object to find the shape of.
            expected_rank: (optional) int. The expected rank of `tensor`. If this is
                specified and the `tensor` has a different rank, and exception will be
                thrown.
            name: Optional name of the tensor for the error message.

        Returns:
            A list of dimensions of the shape of tensor. All static dimensions will
            be returned as python integers, and dynamic dimensions will be returned
            as tf.Tensor scalars.
        """
        if name is None:
            name = tensor.name

        if expected_rank is not None:
            assert_rank(tensor, expected_rank, name)

        shape = tensor.shape.as_list()

        non_static_indexes = []
        for (index, dim) in enumerate(shape):
            if dim is None:
                non_static_indexes.append(index)

        if not non_static_indexes:
            return shape

        dyn_shape = tf.shape(tensor)
        for index in non_static_indexes:
            shape[index] = dyn_shape[index]
        return shape
    
#     config = json.load(open("/data/xuht/bert/chinese_L-12_H-768_A-12/bert_config.json", "r"))
#     init_checkpoint = "/data/xuht/bert/chinese_L-12_H-768_A-12/bert_model.ckpt"
#     config = Bunch(config)
#     config.use_one_hot_embeddings = True
#     config.scope = "bert"
#     config.dropout_prob = 0.1
#     config.label_type = "single_label"
    
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    a = tf.placeholder(tf.float32, [None, None])
    b= tf.range(0, get_shape_list(a, 2)[0])
    sess = tf.Session()
    p = sess.run([b], feed_dict={a:np.random.random((20,10))})
    print(p[0])